In [ ]:
from pyspark.sql.functions import substring_index, col
from pyspark.sql import SparkSession, Row
import re
import pandas as pd
from pyspark.sql import functions as F, Window
import unidecode
from pyspark.sql.functions import isnan, when, count, col, isnull
import matplotlib.pyplot as plt
#############################################
#            read data                      # 
#############################################

file = "/data/SMARTDATA/books/BCT_prestiti/patrons.csv"
DFusers = spark.read.csv(file,header=True)
#DFusers.show(5, False)

file = "/data/SMARTDATA/books/BCT_prestiti/items.csv"
DFitems = spark.read.csv(file,header=True)
# DFitems.show(5, False)

file = "/data/SMARTDATA/books/BCT_prestiti/item_media_types.csv"
DFtypes = spark.read.csv(file,header=True)
# DFtypes.show()

file = "/data/SMARTDATA/books/BCT_prestiti/manifestations.csv"
DFmanifestations = spark.read.csv(file,header=True)

# DFmanifestations = DFmanifestations.filter(DFmanifestations.title.contains("Il mio credo"))
# DFmanifestations.show(5, False)

file = "/data/SMARTDATA/books/BCT_prestiti/loans.csv"
DFloans = spark.read.csv(file,header=True)
#DFmanifestations.show(20, False)

In [30]:
#PLOTS CELL
if False:
    import numpy as np
    from itertools import accumulate
    import matplotlib.pyplot as plt
    from matplotlib import rcParams
    size = 30
    f = 1
    rcParams['figure.figsize'] = 12.6*f, 7*f
    rcParams['xtick.labelsize'] = size
    rcParams['ytick.labelsize'] = size
    rcParams['font.size'] = size
    rcParams["font.sans-serif"] = ["Liberation Sans"]
    rcParams["font.family"] = "sans-serif"
    rcParams.update({'figure.autolayout': True})
    rcParams['pdf.fonttype'] = 42
    rcParams['ps.fonttype'] = 42
    if False:
        #PLOT FOR CDF
        DFloans_grouped = DFloans.groupBy('patron_id_md5').count()
        DFloans_grouped.show(1, False)
        DFloans_pandas = DFloans_grouped.toPandas()['count'].to_list()

        import numpy as np
        data = np.asarray(DFloans_pandas)

        count, bins_count = np.histogram(data, bins=10)
        pdf = count/sum(count)
        cdf = np.cumsum(pdf)
        plt.figure()
        plt.plot(bins_count[1:], cdf, label="CDF")
        plt.legend()
        plt.show()


        #PLOT FOR TYPE OBJECT OCCURRENCES

        DFitems_joined = DFitems.join(DFtypes, DFtypes.item_media_id == DFitems.item_media_id)
        DFitems_total_number = DFitems.count()
        DFitems_types_grouped = DFitems_joined.groupBy('item_media').count()

        list_item_media = DFitems_types_grouped.toPandas()['item_media'].to_list()
        list_counts = DFitems_types_grouped.toPandas()['count'].to_list()

        list_counts = [item/DFitems_total_number for item in list_counts]

        plt.figure()
        plt.bar(list_item_media, list_counts)
        plt.xticks(rotation='vertical')
        plt.ylabel('Occurrences (%)')
        plt.savefig('000_types_occurrences.pdf', bbox_inches='tight')
        plt.close()

        #PLOT FOR BCT SUBSCRIPTIONS
        years = ['2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
        genre_users_years = {}

        for year in years:
            count = DFusers.filter(DFusers.date_created.contains(year)).count()
            genre_users_years[year] = count
        print(genre_users_years)

        names = list(genre_users_years.keys())
        values = list(genre_users_years.values())

        plotting = plt.bar(names, values)
        plt.xticks(rotation='vertical')
        plt.ylabel('Number of subscribed users')
        plt.savefig('001_BCT_subscribers.pdf', bbox_inches='tight')
        plt.close()

        #PLOT FOR BOOKS LANGUAGES (TOP 5)
        file = "/data/SMARTDATA/books/BCT_prestiti/manifestations.csv"
        DFmanifestations = spark.read.csv(file,header=True)
        languages = ['Italian', 'English', 'French', 'German', 'Spanish']
        languages_2 = ['ita', 'eng', 'fre', 'ger', 'spa']

        genre_users_languages = {}

        for language, language2 in zip(languages, languages_2):
            count = DFmanifestations.filter(DFmanifestations.edition_language.contains(language2)).count()
            genre_users_languages[language] = count
        print(genre_users_languages)

        names = list(genre_users_languages.keys())
        values = list(genre_users_languages.values())

        plotting = plt.bar(names, values)
        plt.xticks(rotation='vertical')
        plt.ylabel('Number of books')
        plt.savefig('002_top5_languages_BCT.pdf', bbox_inches='tight')
        plt.close()

        #PLOT FOR BOOKS IN LIBRARIES
        file = "/data/SMARTDATA/books/BCT_prestiti/libraries.csv"
        DFlibraries = spark.read.csv(file,header=True)
        file = "/data/SMARTDATA/books/BCT_prestiti/items.csv"
        DFmanifestations = spark.read.csv(file,header=True)

        DFjoined = DFmanifestations.join(DFlibraries, DFlibraries.library_id == DFmanifestations.home_library_id)
        DFcounting = DFjoined.groupBy('library_name').count().orderBy('count', ascending=False)
        DFcounting.show(10, False)

        DFcounting_pandas = DFcounting.toPandas()

        list_libraries = DFcounting_pandas['library_name'].to_list()
        list_counts = DFcounting_pandas['count'].to_list()

        plotting = plt.bar(list_libraries, list_counts)
        plt.xticks(rotation='vertical')
        plt.ylabel('Number of books')
        plt.savefig('003_book_bct_libraries.pdf', bbox_inches='tight')
        plt.close()

        #CDF loans
        file = "/data/SMARTDATA/books/BCT_prestiti/loans.csv"
        DF = spark.read.csv(file,header=True)
        #print("There are "+str(DF.count())+" ratings")
        prova = DF.groupby(DF.patron_id_md5).count().toPandas()
        #print("BCT the mean is "+str(prova['count'].mean()))
        #print("BCT the median is "+str(prova['count'].median()))
        # USE numpy's HISTOGRAM FUNCTION TO COMPUTE BINS
        xhBCT, xbBCT = np.histogram(prova['count'], bins=6000, normed=True)
        # COMPUTE THE CUMULATIVE SUM WITH accumulate
        xhBCT = list(accumulate(xhBCT))
        # NORMALIZE THE RESULT
        xhBCT = np.array(xhBCT) / max(xhBCT)

        if False:
            file = "/data/SMARTDATA/books/anobii_2021/sql/link_person_item.csv"
            DF = spark.read.csv(file,header=True)
            #print("There are "+str(DF.count())+" ratings")
            prova = DF.groupby(DF.person_id).count().toPandas()
            #print("the mean is "+str(prova['count'].mean()))
            #print("the median is "+str(prova['count'].median()))
            # fig,ax = plt.subplots()
            # USE numpy's HISTOGRAM FUNCTION TO COMPUTE BINS
            xh, xb = np.histogram(prova['count'], bins=6000, normed=True)
            # COMPUTE THE CUMULATIVE SUM WITH accumulate
            xh = list(accumulate(xh))

            # NORMALIZE THE RESULT
            xh = np.array(xh) / max(xh)
        fig,ax = plt.subplots()
        # PLOT WITH LABEL
        #ax.plot(xb[1:], xh, label="Anobii")
        ax.plot(xbBCT[1:], xhBCT, label="BCT")
        #ax.grid()
        #ax.legend()
        ax.set_xlim(-20, 3000)#10000)
        ax.set_ylim(0.0, 1.1)
        #ax.set_title("CDF of Loans per User")
        ax.set_xlabel("Loans per User")
        ax.set_ylabel("CDF")
        #ax.legend(frameon=False)
        plt.savefig("004_cdfLoansPerUserBCT.pdf",\
                            # bbox_extra_artists=(first_legend,second_legend), \
                            bbox_inches='tight')
        plt.close()

/usr/local/share/jupyter/kernels/pyspark_yarn/bdkernelyarn.py:122: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.


In [ ]:
#############################################
#   this part remove from manifestation     #
#              items and loans              #
#    things that are not manuscript and     #
#             monografie                    #
#############################################
DFtypesitems = DFtypes.join(DFitems, DFitems.item_media_id == DFtypes.item_media_id).drop(DFitems.item_media_id)

#Remove in DFitems books of types we are not intersted in
DFitems_filtered_obj = DFitems.filter((DFitems.item_media_id =="B") | (DFitems.item_media_id == "F"))

# tale manifestation_if of books that are kept
manifestation_id_to_keep =  DFitems_filtered_obj.select('manifestation_id').distinct()

#Remove in DFloans loans of books of types we are not intersted in
DFloans_filtered_obj = DFloans.join(DFitems_filtered_obj, DFloans["item_id"] == DFitems_filtered_obj["item_id"], "leftsemi")

#Remove in DFmanifestations books of types we are not intersted in
DFmanifestations_filtered_obj = DFmanifestations.join(manifestation_id_to_keep, \
                                                  DFmanifestations["manifestation_id"] == manifestation_id_to_keep["manifestation_id"], "leftsemi")
# DFmanifestations_filtered.show()
print("Togli oggetti inutili: Prima "+str(DFmanifestations.count())+"  - Dopo "+str(DFmanifestations_filtered_obj.count())) 
#Prima 290125  - Dopo 257054

#############################################
#  this part remove books which are not     #
#                 in italian                #
#       and Topolino and Dylan Dog          #
#############################################
#I checked that the possibilities are ITA or ita for the italian language
#remove in DFmanifestation
DFmanifestations_filtered_ita = DFmanifestations_filtered_obj.filter( (DFmanifestations_filtered_obj.edition_language == "ITA") |\
                                                                 (DFmanifestations_filtered_obj.edition_language == "ita") )
# DFmanifestations_filtered.show()
print("Togli libri in lingua straniera: Prima "+str(DFmanifestations.count())+"  - Dopo "+str(DFmanifestations_filtered_ita.count())) 
DFmanifestations_filtered_comics = DFmanifestations_filtered_ita.filter(~((DFmanifestations_filtered_ita.title.contains("Topolino")) |
                                                               (DFmanifestations_filtered_ita.title.contains("Zagor")) |
                                                               (DFmanifestations_filtered_ita.title.contains("Dylan Dog")) | 
                                                               (DFmanifestations_filtered_ita.title.contains("Tex")) | 
                                                               (DFmanifestations_filtered_ita.title.contains("Paperino")) | 
                                                               (DFmanifestations_filtered_ita.title.contains("Disney   Big"))))

#DFmanifestations_filtered_ita.filter(DFmanifestations_filtered_ita.title.contains("l")).show(20, False)
print("Togli Fumetti: Prima "+str(DFmanifestations.count())+"  - Dopo "+str(DFmanifestations_filtered_comics.count())) 
DFitems_filtered_comics = DFitems_filtered_obj.join(DFmanifestations_filtered_comics, DFmanifestations_filtered_comics["manifestation_id"] == DFitems_filtered_obj["manifestation_id"], \
                                                    "leftsemi")
DFloans_filtered_comics = DFloans_filtered_obj.join(DFitems_filtered_comics, DFloans_filtered_obj["item_id"] == DFitems_filtered_comics["item_id"], \
                                                    "leftsemi")

Togli oggetti inutili: Prima 290125  - Dopo 257054


Togli libri in lingua straniera: Prima 290125  - Dopo 228059


[Stage 248:==========================================>              (3 + 1) / 4]

Togli Fumetti: Prima 290125  - Dopo 227749


In [ ]:
#PULIZIA AUTORI E TITOLI
def mapping(line):
    author_cleaned = line.author #AUTORI CON UN SOLO NOME (DI SOLITO PSEUDONIMI)
    string_value = str(line.ISBNISSN).split(" ")[0]
    ISBNISSN = ''.join(ch for ch in string_value if ch.isalnum())
    if line.author != None:
        if len(line.author.split(" - ")) != 1: #AUTORI MULTIPLI
            author_cleaned = ""
            count = 0
            for author in line.author.split(" - "):
                count += 1
                if len(author.split(", ")) == 2:
                    if count != len(line.author.split(" - ")):
                        author_cleaned += author.split(", ")[1] + " " + author.split(", ")[0] + " - "
                    else:
                        author_cleaned += author.split(", ")[1] + " " + author.split(", ")[0]
                else:
                    if count != len(line.author.split(" - ")):
                        author_cleaned += author.split(", ")[0] + " - "
                    else:
                        author_cleaned += author.split(", ")[0]
        else:
            if len(line.author.split(",")) == 2: #AUTORI CON NOME E COGNOME SEMPLICI
                author_cleaned = line.author.split(", ")[1] + " " + line.author.split(", ")[0]
            
    else: #L'autore è contenuto nel titolo e non nel campo autore
        author_cleaned = ""
        title_zone_temp = line.title.split(" ; ")[0]
        if len(title_zone_temp.split(" / ")) == 2:
            if len(title_zone_temp.split(" / ")[1].split(" e ")) > 1: #AUTORI SEPARATI DA CONGIUNZIONE E
                title_zone = title_zone_temp.split(" / ")[1].split(" e ")
                count = 0
                for part in title_zone:
                    count += 1
                    if count > 1 and count <= len(title_zone):
                        author_cleaned += " - "
                    words = part.split(" ")
                    count_word = 0 
                    for word in words:
                        count_word += 1
                        string_word = str(word)
                        if len(string_word) > 0:
                            if string_word[0].strip(" [").isupper() == True:
                                if count_word == len(words):
                                    author_cleaned += string_word
                                else:
                                    author_cleaned += string_word + " "
                if author_cleaned.startswith(" -"): #NEL CASO CI FOSSE UNA "e" CHE NON SEPARA DUE AUTORI
                    author_cleaned = author_cleaned.replace(" -", "")
            else:
                title_zone = title_zone_temp.split(" / ")[1].replace("[", "").split(" ")
                if len(title_zone) > 0:
                    count = 0
                    for word in title_zone:
                        string_word = str(word)
                        count += 1
                        if len(string_word) > 0:
                            if string_word[0].isupper() == True:
                                if count == len(title_zone):
                                    author_cleaned += string_word
                                else:
                                    author_cleaned += string_word + " "
                        if len(author_cleaned.split(", ")) > 1: #Autori multipli con virgola
                            templist = author_cleaned.split(", ")
                            author_cleaned = ""
                            for author in templist:
                                string_word = str(author)
                                count += 1
                                if len(string_word) > 0:
                                    if string_word[0].isupper() == True:
                                        if count == len(title_zone):
                                            author_cleaned += string_word
                                        else:
                                            author_cleaned += string_word + " - "
    if author_cleaned.startswith(" "):
        author_cleaned = author_cleaned.replace(" ", "", 1)
    author_cleaned = author_cleaned.replace("*", "")
    author_cleaned = author_cleaned.replace('"', "")
    author_cleaned = author_cleaned.replace(":", "")
    author_cleaned = author_cleaned.replace("!", "")
    author_cleaned = author_cleaned.strip(' []*:') #PER PULIRE LE EVENTUALI PARENTESI QUADRE E SPAZI  
    author_cleaned = re.sub('<[^>]+>', '', author_cleaned)
    author_cleaned = author_cleaned.replace("<", "")
    author_cleaned = re.sub(' +',' ', author_cleaned)
    author_cleaned = author_cleaned.replace("_", " ")
    author_cleaned = author_cleaned.replace(">", "").strip(" ")
    
    #NUOVE RIPULITURE SULL'AUTORE
    author_cleaned = author_cleaned.replace("C. Collodi", "Carlo Collodi").strip(" ")
    if author_cleaned == "Collodi":
        author_cleaned = author_cleaned.replace("Collodi", "Carlo Collodi").strip(" ")
    auhtor_cleaned = author_cleaned.replace("; ", " - ")
    author_cleaned = author_cleaned.strip(".")
    author_cleaned = author_cleaned.strip(" -")
    author_cleaned = author_cleaned.strip(")")
    author_cleaned = author_cleaned.replace("...et", "")
    author_cleaned = author_cleaned.replace("[et", "")
    author_cleaned = author_cleaned.strip(".")
    author_cleaned = author_cleaned.replace(",", " - ").strip(" ")
    author_cleaned = author_cleaned.replace(" -  ", " - ").strip(" ")
    author_cleaned = author_cleaned.replace(" -  - ", " - ").strip(" ")
    if author_cleaned == "Teresa Czerniewicz-Umer - Malgorzata OmilanowskaJerzy S. Majewski":
        author_cleaned = "Teresa Czerniewicz-Umer - Malgorzata Omilanowska - Jerzy S. Majewski"
    if author_cleaned == "Vittorio Marsiglia Carlo Missaglia":
        author_cleanded = "Vittorio Marsiglia - Carlo Missaglia"
    if author_cleaned == "Bert Hellinger - Gabriella Gabriele]Hövel":
        author_cleaned = "Bert Hellinger - Gabriella Gabriele Hövel"
    
    #PER ELIMINARE I REFUSI PIù FREQUENTI
    title_cleaned = line.title.split(" / ")[0]
    title_cleaned = title_cleaned.replace("\x89", "").strip(" ")
    title_cleaned = title_cleaned.replace("\x88", "").strip(" ")
    title_cleaned = title_cleaned.replace("\\1", "").strip(" ")
    title_cleaned = title_cleaned.replace("\\2", "").strip(" ")
    title_cleaned = title_cleaned.replace('"', "")
    title_cleaned = title_cleaned.replace('[', "")
    title_cleaned = title_cleaned.replace("\\", "").strip(" ")
    title_cleaned = title_cleaned.replace(']', "")
    title_cleaned = title_cleaned.replace('*', "")
    title_cleaned = title_cleaned.replace('|', "")
    title_cleaned = title_cleaned.replace("!:", "").strip(" ")
    title_cleaned = title_cleaned.replace(" : romanzo", "")
    return Row(line.manifestation_id, line.edition_language, line.edition_date,
               title_cleaned, author_cleaned, line.publisher, ISBNISSN)

def specialcasesmapping(line): #Casi strani disseminati nel Database che riparo manualmente
    new_author = line.author
    new_title = line.title
    new_publisher = line.publisher
    if line.manifestation_id == "440293":
        new_author = "Chiara Lossano"
    if line.manifestation_id == "190617":
        new_author = "Italo Calvino"
    if line.manifestation_id == "430234":
        new_publisher = None
        new_author = "Philip Roth"
        new_title = "«Ho sempre voluto che ammiraste il mio digiuno» ovvero, guardando Kafka"
    if line.manifestation_id == "427622":
        new_author = "Shí Héng Chán - Shì Héng Díng"
    return Row(line.manifestation_id, line.edition_language, line.edition_date,
                   new_title, new_author, line.publisher, line.ISBNISSN)

#clean title and authors
DFmanifestations_filtered_comics = DFmanifestations_filtered_comics.rdd.map(lambda x:mapping(x, ))\
                                .toDF(["manifestation_id","edition_language","edition_date","title","author","publisher", "ISBNISSN"])
DFmanifestations_filtered = DFmanifestations_filtered_comics.rdd.map(lambda x:specialcasesmapping(x, ))\
                                .toDF(["manifestation_id","edition_language","edition_date","title","author","publisher", "ISBNISSN"])
# DFmanifestations_filtered.show(20, False)

#remove row which does not have the author
DFmanifestations_filtered = DFmanifestations_filtered.filter(DFmanifestations_filtered.author != "")

In [ ]:
#############################################
#     this part gives to the same title     # 
#       the same manifestation_id_new       #
#                SOLO TITOLO                #
#############################################

#DFmanifestations_filtered.filter(DFmanifestations_filtered.title == "Il mio credo").show(20, False)
# DFmanifestations_filtered.filter(DFmanifestations_filtered.title == "Il mio credo").show()
DFmanifestations_filtered_title = DFmanifestations_filtered.select(
    "*",
    F.min('manifestation_id').over(Window.partitionBy('title')).alias('manifestation_id_new')
)
# DFmanifestations_filtered_title.filter(DFmanifestations_filtered.title == "Il mio credo").show()

DFmanifestations_filtered_title = DFmanifestations_filtered_title.select(
    "*",
    F.min('ISBNISSN').over(Window.partitionBy('title')).alias('ISBNISSN_new')
)
# DFmanifestations_filtered.show()
# DFmanifestations_filtered_title.filter(DFmanifestations_filtered.title == "Il mio credo").show()

#############################################
#    this part to remove books with less    # 
#              than 50 loans                #
#############################################
#add manifestation_id_new in the item table
# DFitems.join(DFmanifestations_filtered, ['manifestation_id']).show()
 #remove the rows in DFitems but not in DFmanifestation
DFitems_tmp_title = DFitems.join(DFmanifestations_filtered_title, ['manifestation_id'])
#remove useless (in this part) columns
col_to_drop_title = ("manifestation_id", "title", "author", "publisher", "home_library_id", "inventory_date", "edition_language", "item_media_id", "edition_date")
DFitems_dropped_title = DFitems_tmp_title.drop(*col_to_drop_title)
# DFitems_dropped_title.show()

#add manifestation_id_new in the loan table
# DFitems.join(DFmanifestations_filtered, ['manifestation_id']).show()
DFloans_tmp_title = DFloans.join(DFitems_dropped_title, ['item_id'])
# DFloans_tmp.show()

#remove useless (in this part) columns
col_to_drop_title = ("patron_id_md5", "loan_date_begin", "loan_date_end", "due_date", "from_library", "to_library", "end_library")
DFloans_dropped_title = DFloans_tmp_title.drop(*col_to_drop_title)
# DFloans_dropped_title.show()

#groupby sui loans e conteggio prestiti di ogni libro N.B la groupby è fatta sul manifestation_id perchè voglio sapere di un libro e NON di UNA COPIA DI QUEL LIBRO!!!
DFloans_with_item_id_counter_title = DFloans_dropped_title.groupBy("manifestation_id_new").agg(F.count('manifestation_id_new').alias('counter'))#count().alias("counter").show()
# DFloans_with_item_id_counter.show()
# DFloans_with_item_id_counter.filter(DFloans_with_item_id_counter.manifestation_id_new == 118878).show() #sette prestiti

#select only manifestation_id_new with at least 50 loans
DFloans_with_item_id_counter_morethan50_title = DFloans_with_item_id_counter_title.filter(DFloans_with_item_id_counter_title.counter >= 50).select("manifestation_id_new").distinct()
# DFloans_with_item_id_counter_morethan50.show()

#join with DFmanifestation to keep in the manifestation table books with at least 50 loans
DFmanifestations_filtered_aggregated_title = DFmanifestations_filtered_title.join(DFloans_with_item_id_counter_morethan50_title, 
                                                                      DFmanifestations_filtered_title["manifestation_id_new"] == DFloans_with_item_id_counter_morethan50_title["manifestation_id_new"],
                                                                      "leftsemi")

print("prefiltrati: "+str(DFmanifestations_filtered.count())+" - filtrati: "+\
      str(DFmanifestations_filtered_aggregated_title.select(DFmanifestations_filtered_aggregated_title.manifestation_id_new).distinct().count())) #prefiltrati: 218419 - filtrati: 17947



# DFmanifestations_filtered_aggregated.select(DFmanifestations_filtered_aggregated.author).toPandas().to_csv('author.csv')

#############################################
#     this part gives to the same title     # 
#       the same manifestation_id_new       #
#           SOLO TITOLO E AUTORE            #
#############################################
# DFmanifestations_filtered.filter(DFmanifestations_filtered.title == "Il mio credo").show()
column_list = ['title', 'author']
from pyspark.sql.functions import col
DFmanifestations_filtered_title_author = DFmanifestations_filtered.select(
    "*",
    F.min('manifestation_id').over(Window.partitionBy([col(x) for x in column_list])).alias('manifestation_id_new')
)
DFmanifestations_filtered_title_author = DFmanifestations_filtered_title_author.select(
    "*",
    F.min('ISBNISSN').over(Window.partitionBy([col(x) for x in column_list])).alias('ISBNISSN_new')
)
# DFmanifestations_filtered.show()
# DFmanifestations_filtered_title_author.filter(DFmanifestations_filtered.title == "Il mio credo").show()

#############################################
#    this part is to find manifestation     # 
#         with less than 50 loans           #
#############################################
#add manifestation_id_new in the item table
# DFitems.join(DFmanifestations_filtered, ['manifestation_id']).show()
 #remove the rows in DFitems but not in DFmanifestation
DFitems_tmp_title_author = DFitems.join(DFmanifestations_filtered_title_author, ['manifestation_id'])
#remove useless (in this part) columns
col_to_drop_title_author = ("manifestation_id", "title", "author", "publisher", "home_library_id", "inventory_date", "edition_language", "item_media_id", "edition_date")
DFitems_dropped_title_author = DFitems_tmp_title_author.drop(*col_to_drop_title_author)
# DFitems_dropped_title_author.show()

#add manifestation_id_new in the loan table
# DFitems.join(DFmanifestations_filtered, ['manifestation_id']).show()
DFloans_tmp_title_author = DFloans.join(DFitems_dropped_title_author, ['item_id'])
# DFloans_tmp.show()

#remove useless (in this part) columns
col_to_drop_title_author = ("patron_id_md5", "loan_date_begin", "loan_date_end", "due_date", "from_library", "to_library", "end_library")
DFloans_dropped_title_author = DFloans_tmp_title_author.drop(*col_to_drop_title_author)
# DFloans_dropped_title_author.show()

#groupby sui loans e conteggio prestiti di ogni libro N.B la groupby è fatta sul manifestation_id perchè voglio sapere di un libro e NON di UNA COPIA DI QUEL LIBRO!!!
DFloans_with_item_id_counter_title_author = DFloans_dropped_title_author.groupBy("manifestation_id_new").agg(F.count('manifestation_id_new').alias('counter'))#count().alias("counter").show()
# DFloans_with_item_id_counter.show()
# DFloans_with_item_id_counter.filter(DFloans_with_item_id_counter.manifestation_id_new == 118878).show() #sette prestiti

22/11/19 23:22:32 WARN sql.Column: Constructing trivially true equals predicate, 'manifestation_id_new#2219 = manifestation_id_new#2219'. Perhaps you need to use aliases.
[Stage 229:===================================================> (196 + 4) / 200]

prefiltrati: 218419 - filtrati: 17947


In [41]:
#DEFAULT THRESHOLD 100 LOANS
#list_books = []
#for i in [20, 40, 60, 80, 100, 120, 140]: #FOR LOANS THRESHOLD PLOT


#select only manifestation_id_new with at least 50 loans
DFloans_with_item_id_counter_morethan50_title_author = DFloans_with_item_id_counter_title_author.filter(DFloans_with_item_id_counter_title_author.counter >= 100).select("manifestation_id_new").\
                                                       distinct()
# DFloans_with_item_id_counter_morethan50.show()

#join with DFmanifestation to keep in the manifestation table books with at least 50 loans
DFmanifestations_filtered_aggregated_title_author = DFmanifestations_filtered_title_author.join(DFloans_with_item_id_counter_morethan50_title_author, 
                                                                      DFmanifestations_filtered_title_author["manifestation_id_new"] == DFloans_with_item_id_counter_morethan50_title_author["manifestation_id_new"],
                                                                      "leftsemi")
list_books.append(DFmanifestations_filtered_aggregated_title_author.count())
print("Manifestation prefiltrati: "+str(DFmanifestations_filtered.count())+" - filtrati: "+\
      str(DFmanifestations_filtered_aggregated_title_author.select(DFmanifestations_filtered_aggregated_title_author.manifestation_id_new).distinct().count())) #prefiltrati: 218419 - filtrati: 18164

# DFmanifestations_filtered_aggregated.select(DFmanifestations_filtered_aggregated.author).toPandas().to_csv('author.csv')

22/12/02 19:39:31 WARN sql.Column: Constructing trivially true equals predicate, 'manifestation_id_new#5021 = manifestation_id_new#5021'. Perhaps you need to use aliases.
22/12/02 19:40:32 WARN sql.Column: Constructing trivially true equals predicate, 'manifestation_id_new#5021 = manifestation_id_new#5021'. Perhaps you need to use aliases.


Manifestation prefiltrati: 218419 - filtrati: 42723


22/12/02 19:41:43 WARN sql.Column: Constructing trivially true equals predicate, 'manifestation_id_new#5021 = manifestation_id_new#5021'. Perhaps you need to use aliases.


Manifestation prefiltrati: 218419 - filtrati: 22807


22/12/02 19:43:01 WARN sql.Column: Constructing trivially true equals predicate, 'manifestation_id_new#5021 = manifestation_id_new#5021'. Perhaps you need to use aliases.


...

Manifestation prefiltrati: 218419 - filtrati: 7621


22/12/02 19:46:22 WARN sql.Column: Constructing trivially true equals predicate, 'manifestation_id_new#5021 = manifestation_id_new#5021'. Perhaps you need to use aliases.


Manifestation prefiltrati: 218419 - filtrati: 5719


[Stage 676:=================================================>   (187 + 5) / 200]

Manifestation prefiltrati: 218419 - filtrati: 4365


In [43]:
if False:
    #PLOT BOOKS PER LOANS THRESHOLD
    plt.figure()
    plotting = plt.plot([20,40,60,80,100,120,140], list_books)
    plt.xlabel('Loans threshold')
    plt.ylabel('Number of books')
    plt.savefig('005_loans_books_threshold.pdf', bbox_inches='tight')
    plt.close()

In [23]:
#to verify how many null and nan are present in the ISBNISSN_new, ISBNISSN field anymore
# DFmanifestations_filtered_aggregated_title_author.select([count(when(isnan(c) | isnull(c), c)).alias(c) for c in ["ISBNISSN"]]).show()
# DFmanifestations_filtered_aggregated_title_author.select([count(when(isnan(c) | isnull(c), c)).alias(c) for c in ["ISBNISSN_new"]]).show()
# DFmanifestations.select([count(when(isnan(c) | isnull(c), c)).alias(c) for c in ["ISBNISSN"]]).show()

DFmanifestations_final = DFmanifestations_filtered_aggregated_title_author
DFitems_final = DFitems_filtered_comics.join(DFmanifestations_final, DFmanifestations_final["manifestation_id"] == DFitems_filtered_comics["manifestation_id"], \
                                                    "leftsemi")
DFloans_final = DFloans_filtered_comics.join(DFitems_final, DFloans_filtered_comics["item_id"] == DFitems_final["item_id"], \
                                                    "leftsemi")

print("Item prefiltrati: "+str(DFitems.count())+" - filtrati: "+\
      str(DFitems_final.count())) #prefiltrati: 998403 - filtrati: 217405
print("Loan prefiltrati: "+str(DFloans.count())+" - filtrati: "+\
      str(DFloans_final.count())) #prefiltrati: 5484078 - filtrati: 2172971

Item prefiltrati: 998403 - filtrati: 114921


[Stage 338:=================================================>   (128 + 9) / 137]

Loan prefiltrati: 5484078 - filtrati: 1445877


In [24]:
#substitute manifestation_id with manifestation_id_new
DFitems_definitive = DFitems_final.join(DFmanifestations_final,DFmanifestations_final["manifestation_id"] == DFitems_final["manifestation_id"], "inner")\
                                  .select(DFmanifestations_final.manifestation_id_new, \
                                          DFitems_final.item_id, \
                                          DFitems_final.item_media_id,\
                                          DFitems_final.home_library_id,\
                                          DFitems_final.inventory_date)
# DFitems_definitive.show(2, False)
#substitute manifestation_id with manifestation_id_new
DFloans_definitive = DFloans_final.join(DFitems_definitive, DFitems_definitive["item_id"] == DFloans_final["item_id"], "inner")\
                                  .select(DFitems_definitive.manifestation_id_new, \
                                          DFloans_final.patron_id_md5, \
                                          DFloans_final.loan_date_begin,\
                                          DFloans_final.loan_date_end,\
                                          DFloans_final.due_date,\
                                          DFloans_final.from_library,\
                                          DFloans_final.to_library,\
                                          DFloans_final.end_library,)
# DFloans_definitive.show(2, False)

#DFmanifestations_definitive contains only manifestation_id and ISBNISSN
DFmanifestations_definitive = DFmanifestations_final.groupby("manifestation_id_new").agg(F.min("manifestation_id").alias("manifestation_id"),\
                                                                                         F.min("ISBNISSN").alias("ISBNISSN")).drop("manifestation_id_new").drop("ISBNISSN_new")#\
# DFmanifestations_final.show(2, False)
# DFmanifestations_definitive.show(2, False)
#per avere tutti gli attributi in DFmanifestations_definitive e togliere manifestation_id e ISBNISSN e tenere solo manifestation_id_new e ISBNISSN_new 
DFmanifestations_definitive = DFmanifestations_definitive.join(DFmanifestations_final, DFmanifestations_final["manifestation_id"] == DFmanifestations_definitive["manifestation_id"])\
                                                         .drop(DFmanifestations_final["manifestation_id"]).drop(DFmanifestations_definitive["manifestation_id"])\
                                                         .drop(DFmanifestations_final["ISBNISSN"]).drop(DFmanifestations_definitive["ISBNISSN"])
# DFmanifestations_definitive.show(2, False)
print("i libri sono: "+str(DFmanifestations_definitive.count())) #--> i libri sono: 18164
# DFmanifestations_definitive.count() deve essere uguale a DFmanifestations_final.select("manifestation_id_new").distinct().count()

[Stage 348:====================================================>(198 + 2) / 200]

i libri sono: 7621


In [7]:
# DFmanifestations_final.filter(DFmanifestations_final.title == "Pinocchio").show(3, False)
# DFmanifestations_definitive.filter(DFmanifestations_definitive.title == "Pinocchio").show(3, False)

+----------------+----------------+------------+---------+-------------+-----------+-------------+--------------------+------------+
|manifestation_id|edition_language|edition_date|title    |author       |publisher  |ISBNISSN     |manifestation_id_new|ISBNISSN_new|
+----------------+----------------+------------+---------+-------------+-----------+-------------+--------------------+------------+
|658351          |ita             |2016        |Pinocchio|Carlo Collodi|Gribaudo   |9788858014554|107930              |8807820714  |
|193757          |ita             |1972        |Pinocchio|Carlo Collodi|Feltrinelli|None         |107930              |8807820714  |
|314571          |ita             |2006        |Pinocchio|Carlo Collodi|Giunti kids|9788809046085|107930              |8807820714  |
+----------------+----------------+------------+---------+-------------+-----------+-------------+--------------------+------------+
only showing top 3 rows

+----------------+------------+---------+---

In [9]:
#SAVE FILES
DFmanifestations_definitive.toPandas().to_csv("manifestationCleaned.csv")
DFloans_definitive.toPandas().to_csv("loansCleaned.csv")